In [1]:
import torch

x = torch.ones(5)  
y = torch.zeros(3)  
w = torch.randn(5, 3, requires_grad=True)
b = torch.randn(3, requires_grad=True)
z = torch.matmul(x, w)+b
loss = torch.nn.functional.binary_cross_entropy_with_logits(z, y)

# w, b는 최적화가 필요한 파라미터이기 때문에 이에 대한 손실함수의 기울기를 계산해야 함. 
# 그래서 w, b에 requires_grad() 메서드가 필요

Function class에는 순전파 방향으로 함수를 계산하는 방법 & 역전파 단계에서 그 함수의 도함수를 계산하는 속성이 있다. grad_fn을 통해 역전파 단계 함수에 대한 grad를 확인할 수 있다.

In [2]:
print(f"Gradient function for z = {z.grad_fn}")
print(f"Gradient function for loss = {loss.grad_fn}")

Gradient function for z = <AddBackward0 object at 0x104691750>
Gradient function for loss = <BinaryCrossEntropyWithLogitsBackward0 object at 0x104691780>


In [3]:
loss.backward()
print(w.grad)
print(b.grad)

tensor([[0.3187, 0.2307, 0.1760],
        [0.3187, 0.2307, 0.1760],
        [0.3187, 0.2307, 0.1760],
        [0.3187, 0.2307, 0.1760],
        [0.3187, 0.2307, 0.1760]])
tensor([0.3187, 0.2307, 0.1760])


### Disabling Gradient Tracking
모델이 이미 훈련되었고(모델 파라미터 고정), 입력 데이터를 적용해 순전파 계산만 하고 싶다면(연산 시간 감소) 기울기 추적 필요 X

=> 이런 경우 torch.no_grad() 블록으로 계산 코드를 감싸서 계산 추적 중지할 수 있다.

1. torch.no_grad()

In [4]:
z = torch.matmul(x, w)+b
print(z.requires_grad)

with torch.no_grad():
    z = torch.matmul(x, w)+b
print(z.requires_grad)

True
False


2. detach()

In [5]:
z = torch.matmul(x, w)+b
z_det = z.detach()
print(z_det.requires_grad)

False


---

In [6]:
inp = torch.eye(4, 5, requires_grad=True)
out = (inp+1).pow(2).t()
out.backward(torch.ones_like(out), retain_graph=True)
print(f"First call\n{inp.grad}")
out.backward(torch.ones_like(out), retain_graph=True)
print(f"\nSecond call\n{inp.grad}")
inp.grad.zero_()
out.backward(torch.ones_like(out), retain_graph=True)
print(f"\nCall after zeroing gradients\n{inp.grad}")

First call
tensor([[4., 2., 2., 2., 2.],
        [2., 4., 2., 2., 2.],
        [2., 2., 4., 2., 2.],
        [2., 2., 2., 4., 2.]])

Second call
tensor([[8., 4., 4., 4., 4.],
        [4., 8., 4., 4., 4.],
        [4., 4., 8., 4., 4.],
        [4., 4., 4., 8., 4.]])

Call after zeroing gradients
tensor([[4., 2., 2., 2., 2.],
        [2., 4., 2., 2., 2.],
        [2., 2., 4., 2., 2.],
        [2., 2., 2., 4., 2.]])
